In [1]:
# 查看当前挂载的数据集目录
!ls /home/kesci/input/

lmy05287549


In [2]:
# 查看个人持久化工作区文件
!ls /home/kesci/work/

lllmy_province.csv  lmy_tu.csv	    provincial_flows_1.csv  spark-warehouse
lmy_Province.csv    lmy_tu.parquet  provincial_flows_2.csv


In [ ]:
# 查看当前kernel下的package
!pip list --format=columns

In [ ]:
# 显示cell运行时长
%load_ext klab-autotime

In [3]:
import pandas as pd
import findspark
import pyspark
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import when,udf
from pyspark.sql.types import DateType , IntegerType,StringType
spark = SparkSession.builder.getOrCreate()

In [4]:
#读入parquet数据
df = spark.read.format("parquet").load('/home/kesci/input/lmy05287549/*.parquet')
df.show()
df.count()

+--------+-------------------+----------------+----------+--------+----------+------------------+----------+----------+----------+------------------+----------+----------+--------+
|班次代码|           发车时间|      乘车站名称|到达站名称|座位类型|乘车站省份|乘车站行政地理分区|乘车站经度|乘车站纬度|到达站省份|到达站行政地理分区|到达站经度|到达站纬度|    距离|
+--------+-------------------+----------------+----------+--------+----------+------------------+----------+----------+----------+------------------+----------+----------+--------+
|  KS1057|2020-05-07 01:00:00|苏州北广场汽车站|  常熟南站|       1|    江苏省|              华东|120.608475| 31.330946|    江苏省|              华东| 120.74239| 31.628862|  35.478|
|  KS3197|2020-05-07 02:50:00|        苏州南站|  常熟南站|       1|    江苏省|              华东|120.638145|  31.27728|    江苏省|              华东| 120.74239| 31.628862|  40.325|
|  GT1001|2020-05-07 09:40:00|          沙溪站|    太仓站|       1|    湖南省|              华中|109.902885| 26.756468|    江苏省|              华东| 121.19665| 31.510124| 1216.72|
|  GT1001|2020-05-07 09:40:00|          沙溪

38448537

In [5]:
#将需要在sql中用到的变量重新命名
df = df.withColumnRenamed("乘车站名称",'StartStation')
df = df.withColumnRenamed("到达站名称",'ReachStation')
df = df.withColumnRenamed("乘车站省份",'StartProvince')
df = df.withColumnRenamed("到达站省份",'ReachProvince')
df = df.withColumnRenamed("乘车站经度",'StartLongitude')
df = df.withColumnRenamed("乘车站纬度",'StartLatitude')
df = df.withColumnRenamed("到达站经度",'ReachLongitude')
df = df.withColumnRenamed("到达站纬度",'ReachLatitude')
df = df.withColumnRenamed("乘车站行政地理分区",'StartAD')#administrative divisions
df = df.withColumnRenamed("到达站行政地理分区",'ReachAD')
df = df.withColumnRenamed("座位类型",'Seat')
df.show(5)

+--------+-------------------+----------------+------------+----+-------------+-------+--------------+-------------+-------------+-------+--------------+-------------+-------+
|班次代码|           发车时间|    StartStation|ReachStation|Seat|StartProvince|StartAD|StartLongitude|StartLatitude|ReachProvince|ReachAD|ReachLongitude|ReachLatitude|   距离|
+--------+-------------------+----------------+------------+----+-------------+-------+--------------+-------------+-------------+-------+--------------+-------------+-------+
|  KS1057|2020-05-07 01:00:00|苏州北广场汽车站|    常熟南站|   1|       江苏省|   华东|    120.608475|    31.330946|       江苏省|   华东|     120.74239|    31.628862| 35.478|
|  KS3197|2020-05-07 02:50:00|        苏州南站|    常熟南站|   1|       江苏省|   华东|    120.638145|     31.27728|       江苏省|   华东|     120.74239|    31.628862| 40.325|
|  GT1001|2020-05-07 09:40:00|          沙溪站|      太仓站|   1|       湖南省|   华中|    109.902885|    26.756468|       江苏省|   华东|     121.19665|    31.510124|1216.72|
|  GT1001|

In [6]:
#创建数据本地临时视图
df.createOrReplaceTempView('data')

In [7]:
#利用sql语句按照车站汇总车次
#SQL语句
query = """
select row_number() over(order by num desc) as id, Station,Province,Longitude,Latitude,num
from (
    select Station,Province,Longitude,Latitude,sum(num) as num from
    (
        (select StartStation as Station,StartProvince as Province,
        StartLongitude as Longitude,StartLatitude  as Latitude,
        count(*) as num from data 
        group by StartStation,StartProvince,StartLongitude,StartLatitude
        order by num desc) 
        union
        (select ReachStation  as Station,  ReachProvince as Province,
        ReachLongitude as Longitude,ReachLatitude  as Latitude,
        count(*) as num from data 
        group by ReachStation,ReachProvince,ReachLongitude,ReachLatitude
        order by num desc) 
    )
        group by Station,Province,Longitude,Latitude
        order by num desc
    )
"""
Station = spark.sql(query)
Station.show(10)
#创建视图
Station.createOrReplaceTempView('Station')

+---+------------+--------+----------+---------+-------+
| id|     Station|Province| Longitude| Latitude|    num|
+---+------------+--------+----------+---------+-------+
|  1|    苍南西站|  浙江省| 120.38268| 27.49879|1230838|
|  2|        温州|  浙江省| 120.69936|27.994267|1196258|
|  3|  义乌火车站|  浙江省|120.042786|29.377123| 973030|
|  4|    东阳东站|  浙江省| 120.25049|29.259472| 929119|
|  5|  无锡客运站|  江苏省| 120.30845|31.592854| 616847|
|  6|    萧山机场|  浙江省| 120.43706|30.234344| 613546|
|  7|    东阳西站|  浙江省| 120.18615|29.298126| 558542|
|  8|  东阳横店站|  浙江省| 120.20354| 29.28787| 505864|
|  9|    南通东站|  江苏省| 120.89203|32.004845| 460216|
| 10|虹桥长途西站|  江苏省|120.845825| 32.00245| 430529|
+---+------------+--------+----------+---------+-------+
only showing top 10 rows



In [8]:
query = """
select a.Station as StartStation,b.Station as ReachStation, t.num
from (
select StartStation as src,ReachStation as dst,count(*) as num
from data
group by StartStation,ReachStation order by num desc ) as t,Station as a,Station as b

where t.src = a.Station and t.dst = b.Station
order by t.num desc
"""
Station_to_Station = spark.sql(query)
Station_to_Station.createOrReplaceTempView('Station_to_Station')
Station_to_Station.show(10)

+------------------+-------------------+-------+
|      StartStation|       ReachStation|    num|
+------------------+-------------------+-------+
|          苍南西站|               温州|1101392|
|          东阳东站|         义乌火车站| 744609|
|          东阳西站|             杭州东| 252622|
|        东阳横店站|         义乌火车站| 216571|
|        东阳横店站|           萧山机场| 191131|
|        昌吉客运站|     乌鲁木齐高铁站| 136342|
|        岱山长途站|             宁波南| 136089|
|        中川机场站|兰州火车站（1号线）| 132122|
|          东阳西站|           萧山机场| 127423|
|乌鲁木齐高铁汽车站|         昌吉新客站| 123830|
+------------------+-------------------+-------+
only showing top 10 rows



In [9]:
#下面是以省为单位进行分类汇总

In [10]:
query = """
select Province,station_Longitude_scatter,station_Latitude_scatter,num,station_num from(
    select Province,
    STDDEV(Longitude) as station_Longitude_scatter ,
    STDDEV(Latitude) as station_Latitude_scatter,
    sum(num) as num,
    count(*) as station_num
    from Station
    group by Province
    )
    order by num desc
"""
res_province = spark.sql(query)
res_province.show(32)

+----------------+-------------------------+------------------------+--------+-----------+
|        Province|station_Longitude_scatter|station_Latitude_scatter|     num|station_num|
+----------------+-------------------------+------------------------+--------+-----------+
|          江苏省|       1.0181876204590883|      0.9929181389794784|15511581|       1759|
|          浙江省|       0.8746601761423658|      0.8953315777361526|13312965|       1672|
|          辽宁省|       1.5478593076652218|      0.8918700260517638| 7464466|       2540|
|          四川省|       1.5776112682629657|       1.308219043939877| 5387613|       1904|
|          贵州省|       1.2774191423452064|      0.9750793504995253| 5357614|       1094|
|          广东省|       1.4663641024726444|      0.8213795250488088| 3161787|       3207|
|          陕西省|       1.0835063215033525|      1.4754995486523017| 2912935|       1221|
|          湖南省|       1.2012502647183145|      1.2002256216924627| 2390135|       1278|
|          湖北省|       1

In [13]:
res_province.createOrReplaceTempView('res_province')

In [11]:
query = """
select StartProvince as Province,
sum(case when data.Seat=1 then 1 else 0 end) as sseat_1,
sum(case when data.Seat=2 then 1 else 0 end) as sseat_2,
sum(case when data.Seat=3 then 1 else 0 end) as sseat_3,
sum(case when data.Seat=4 then 1 else 0 end) as sseat_4,
sum(case when data.Seat=9 then 1 else 0 end) as sseat_9
from data
group by StartProvince
"""
StartStation_seat = spark.sql(query)
StartStation_seat.show()
StartStation_seat.createOrReplaceTempView('StartStation_seat')

+--------------+-------+-------+-------+-------+-------+
|      Province|sseat_1|sseat_2|sseat_3|sseat_4|sseat_9|
+--------------+-------+-------+-------+-------+-------+
|    西藏自治区| 589329|  87779|      0|      0|    687|
|        北京市|    941|      0|      0|      0|    445|
|        辽宁省|4078812|     42|      0|      0|   3059|
|        浙江省|6690276|     56|      0|      0|  21443|
|广西壮族自治区|  45256|    630|    233|      0|  38791|
|        海南省| 518404|      0|      0|      0|    286|
|        重庆市|  28906|      0|      0|      0|      0|
|        河北省|  16532|      0|      0|      0|      0|
|        福建省| 148561|      0|      0|      0|      0|
|        湖南省|1223240|      0|      0|      0|  41286|
|宁夏回族自治区| 723933|      0|      0|      0|      0|
|        陕西省|1471044|      1|      0|      0|   3218|
|        山西省|  77866|      0|      0|      0|      0|
|  内蒙古自治区| 312366|    182|     23|     86|   1386|
|        甘肃省| 735983|    995|      0|      0| 249823|
|        贵州省|2935917|   2347|   

In [12]:
query = """
select ReachProvince as Province,
sum(case when data.Seat=1 then 1 else 0 end) as rseat_1,
sum(case when data.Seat=2 then 1 else 0 end) as rseat_2,
sum(case when data.Seat=3 then 1 else 0 end) as rseat_3,
sum(case when data.Seat=4 then 1 else 0 end) as rseat_4,
sum(case when data.Seat=9 then 1 else 0 end) as rseat_9
from data
group by ReachProvince
"""
ReachStation_seat = spark.sql(query)
ReachStation_seat.show()
ReachStation_seat.createOrReplaceTempView('ReachStation_seat')

+--------------+-------+-------+-------+-------+-------+
|      Province|rseat_1|rseat_2|rseat_3|rseat_4|rseat_9|
+--------------+-------+-------+-------+-------+-------+
|    西藏自治区| 575251|  87636|      0|      0|   1064|
|        北京市|  74442|    182|      3|     68|    351|
|        辽宁省|3380770|     26|      0|      0|   1757|
|        浙江省|6501181|     12|      7|      0|  99990|
|广西壮族自治区| 244635|    520|      9|      0|  38513|
|        海南省| 547459|     14|      1|      0|    223|
|        重庆市| 289235|    852|      0|      0|   7687|
|香港特别行政区|  19215|      0|      1|      0|     14|
|        河北省| 203210|      0|     20|     18|   1649|
|        福建省| 314379|      1|      4|      0|    714|
|        湖南省|1107005|     65|      1|      0|  18544|
|宁夏回族自治区| 644169|      0|      0|      0|   8105|
|        台湾省|    375|      0|      0|      0|      0|
|        陕西省|1428298|    285|      2|      0|  10087|
|澳门特别行政区|     29|      0|      0|      0|      0|
|        山西省| 296724|      5|     10|

In [14]:
res_final = res_province.join(StartStation_seat,on="Province")
res_final = res_final.join(ReachStation_seat,on="Province")
res_final.show()

+--------------+-------------------------+------------------------+--------+-----------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|      Province|station_Longitude_scatter|station_Latitude_scatter|     num|station_num|sseat_1|sseat_2|sseat_3|sseat_4|sseat_9|rseat_1|rseat_2|rseat_3|rseat_4|rseat_9|
+--------------+-------------------------+------------------------+--------+-----------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|    西藏自治区|        3.896589125146099|       1.361767083992596| 1341746|        257| 589329|  87779|      0|      0|    687| 575251|  87636|      0|      0|   1064|
|        北京市|       0.3329380840313544|     0.29696166439394955|   76432|        216|    941|      0|      0|      0|    445|  74442|    182|      3|     68|    351|
|        辽宁省|       1.5478593076652218|      0.8918700260517638| 7464466|       2540|4078812|     42|      0|      0|   3059|3380770|     26|      0|      0|   175